In [ ]:
from utils.helpers import *
import pandas as pd
import numpy as np

In [5]:
np.random.seed(0)

### Risk Model

Let's assume the following risk model using which we will be computing various risk-based portfolios:

$$\Sigma = \beta\Omega\beta' + D $$ 

In [6]:
# factor loadings
beta = np.array([[0.9,0,0.5],
                 [1.1,0.5,0], 
                 [1.2,.3,.2],
                 [.8,.1,.7]])

# factor covariance matrix
factor_sigma = np.array([.2,.1,.1])
factor_corr = np.identity(len(factor_sigma))
Omega = factor_corr * (factor_sigma[:,None] @ factor_sigma[:,None].T)

# idiosyncratic covariance matrix
idio_vols = np.array([.1,.15,.1,.15])
D = np.diag(idio_vols**2)

# asset covariance matrix
Sigma = beta@Omega@beta.T+D

# inverse matrices (to avoid computation over again, useful for factor risk contributions of a given portfolio)
pseudo_inv_beta = beta@np.linalg.inv(beta.T@beta)
inv_cov = np.linalg.inv(Sigma)
matrix_min_vol = inv_cov@beta@np.linalg.inv(beta.T@inv_cov@beta) 

d, m = beta.shape

if np.linalg.matrix_rank(beta)!=m:
    raise ValueError('The factor loadings matrix is NOT full rank!')

if is_pos_def(Sigma)==False:
    raise ValueError('The covariance matrix is NOT PSD!')

print('======================================================')
print('Number of assets:', d)
print('Number of factors:', m)
print('======================================================')
print('Betas:', beta)
print('======================================================')
print('Covariance matrix:', Sigma)

Number of assets: 4
Number of factors: 3
Betas: [[0.9 0.  0.5]
 [1.1 0.5 0. ]
 [1.2 0.3 0.2]
 [0.8 0.1 0.7]]
Covariance matrix: [[0.0449 0.0396 0.0442 0.0323]
 [0.0396 0.0734 0.0543 0.0357]
 [0.0442 0.0543 0.0689 0.0401]
 [0.0323 0.0357 0.0401 0.0531]]


### Risk Budgeting (RB) portfolio

Optimization problem to find the portfolio whose assets contribute to the total risk (volatility) in proportion to specified risk budgets: 
$$ \min_{y \in \mathbb (R_+^*)^d} y'\Sigma y - \sum_{i=1}^d b_i \log{y_i}$$ 

In [7]:
### ERC portfolio
df_erc = pd.DataFrame(index=['Asset 1','Asset 2','Asset 3','Asset 4','Factor 1','Factor 2','Factor 3'],
                     columns=['Exposure', 'Risk Contribution'], dtype='float')


### Optimization problem for ERC portfolio
# asset risk budgets (chosen equal)
budgets = np.ones(d)/d
_, theta_erc = compute_risk_budgeting_portfolio(budgets, Sigma)

# factor allocations of the computed portfolio
w_erc = theta_erc@beta

df_erc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Exposure'] = theta_erc
df_erc.loc[['Factor 1','Factor 2','Factor 3'],'Exposure'] = w_erc

df_erc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Risk Contribution'] =  asset_vol_contribution(theta_erc, Sigma)
df_erc.loc[['Factor 1','Factor 2','Factor 3'],'Risk Contribution'] =  w_erc*grad_factor_risk_measure_volatility_analytical(w_erc, matrix_min_vol, pseudo_inv_beta, Sigma)

df_erc.columns.name = 'ERC'

### Factor RB portfolio

Optimization problem to find the portfolio whose factors contribute to the total risk (volatility) in proportion to specified risk budgets: 

$$\min_{y \in \mathcal C} y'\Sigma y - \sum_{i=1}^m b_i \log{(\beta'y)_i}$$

where $\mathcal C = \big\{y\in\mathbb{R}^d | (\beta'y)_i>0, \forall i \in \{1,\ldots,m\}\big\}$.

In [8]:
### Factor ERC portfolio
df_ferc = pd.DataFrame(index=['Asset 1','Asset 2','Asset 3','Asset 4','Factor 1','Factor 2','Factor 3'],
                     columns=['Exposure', 'Risk Contribution'], dtype='float')

### Optimization problem for Factor ERC portfolio
# factor risk budgets (chosen equal)
factor_budgets = np.ones(m)/m
_, theta_ferc = compute_factor_risk_budgeting_portfolio(factor_budgets, beta, Sigma)

# factor allocations of the computed portfolio
w_ferc = theta_ferc@beta

df_ferc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Exposure'] = theta_ferc
df_ferc.loc[['Factor 1','Factor 2','Factor 3'],'Exposure'] = w_ferc

df_ferc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Risk Contribution'] =  asset_vol_contribution(theta_ferc, Sigma)
df_ferc.loc[['Factor 1','Factor 2','Factor 3'],'Risk Contribution'] =  w_ferc*grad_factor_risk_measure_volatility_analytical(w_ferc, matrix_min_vol, pseudo_inv_beta, Sigma)

df_ferc.columns.name = 'FERC'

### Asset-Factor RB portfolio

Proposed optimization problem to find the portfolio whose assets and factors contribute to the total risk (volatility) in a balanced way while remaining close to the specified risk budgets:

$$ \min_{y \in \mathcal C^{> 0}} y'\Sigma y  -  \lambda_a \sum_{i=1}^d {b_a}_i \log{y_i} - \lambda_f \sum_{i=1}^m {b_f}_i \log{(\beta'y)_i}$$
where $\mathcal C^{>0} = \big\{y\in(\mathbb R_+^*)^d | (\beta'y)_i>0, \forall i \in \{1,\ldots,m\}\big\}$ and $\lambda_a, \lambda_f \in \mathbb R_+^*$ are asset and factor importance parameters. 

In [9]:
### Asset-Factor ERC portfolios
df_aferc = pd.DataFrame(index=['Asset 1','Asset 2','Asset 3','Asset 4','Factor 1','Factor 2','Factor 3'],
                     columns=['Exposure', 'Risk Contribution'], dtype='float')

### Optimization problem for Asset-Factor ERC portfolio
# factor risk budgets (chosen equal) and asset risk budgets (chosen equal)
factor_budgets = np.ones(m)/m
asset_budgets = np.ones(d)/d

# factor and asset "importance" parameters
factor_coef = .8
asset_coef = .2

_, theta_aferc = compute_asset_factor_risk_budgeting_portfolio(asset_budgets,
                                                               factor_budgets,
                                                               asset_coef,
                                                               factor_coef,
                                                               beta,
                                                               Sigma)

# factor allocations of the computed portfolio
w_aferc = theta_aferc@beta

df_aferc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Exposure'] = theta_aferc
df_aferc.loc[['Factor 1','Factor 2','Factor 3'],'Exposure'] = w_aferc

df_aferc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Risk Contribution'] =  asset_vol_contribution(theta_aferc, Sigma)
df_aferc.loc[['Factor 1','Factor 2','Factor 3'],'Risk Contribution'] =  w_aferc*grad_factor_risk_measure_volatility_analytical(w_aferc, matrix_min_vol, pseudo_inv_beta, Sigma)

df_aferc.columns.name = 'AFERC'

### Nearest Asset-Factor RB portfolios

We propose a framework that automatically calibrates the importance parameters $\lambda_a$ and $\lambda_f$ using Bayesian optimization, aiming to minimize the mean distance between asset and factor risk contributions and their respective budgets.

In [10]:
### Nearest Asset-Factor ERC portfolios
df_naferc = pd.DataFrame(index=['Asset 1','Asset 2','Asset 3','Asset 4','Factor 1','Factor 2','Factor 3'],
                     columns=['Exposure', 'Risk Contribution'], dtype='float')

### Optimization problem for Nearest Asset-Factor ERC portfolio
# factor risk budgets (chosen equal) and asset risk budgets (chosen equal)
factor_budgets = np.ones(m)/m
asset_budgets = np.ones(d)/d

theta_naferc, opt_lambda_a = compute_nearest_asset_factor_risk_budgeting_portfolio(asset_budgets, 
                                                                                   factor_budgets, 
                                                                                   beta,
                                                                                   Sigma)

# factor allocations of the computed portfolio
w_naferc = theta_naferc@beta

df_naferc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Exposure'] = theta_naferc
df_naferc.loc[['Factor 1','Factor 2','Factor 3'],'Exposure'] = w_naferc

df_naferc.loc[['Asset 1','Asset 2','Asset 3','Asset 4'],'Risk Contribution'] =  asset_vol_contribution(theta_naferc, Sigma)
df_naferc.loc[['Factor 1','Factor 2','Factor 3'],'Risk Contribution'] =  w_naferc*grad_factor_risk_measure_volatility_analytical(w_naferc, matrix_min_vol, pseudo_inv_beta, Sigma)

df_naferc.columns.name = 'NAFERC'

[I 2025-03-24 16:26:41,803] A new study created in memory with name: no-name-bd9e7f24-576b-4b43-8c23-93c27639ea57
[I 2025-03-24 16:26:41,820] Trial 0 finished with value: 0.8824919071488326 and parameters: {'lamdba_a': 3.668687240870525}. Best is trial 0 with value: 0.8824919071488326.
[I 2025-03-24 16:26:41,839] Trial 1 finished with value: 0.7268400674932671 and parameters: {'lamdba_a': 0.724359113803278}. Best is trial 1 with value: 0.7268400674932671.
[I 2025-03-24 16:26:41,854] Trial 2 finished with value: 0.8691913158797797 and parameters: {'lamdba_a': 2.8751806146011383}. Best is trial 1 with value: 0.7268400674932671.
[I 2025-03-24 16:26:41,865] Trial 3 finished with value: 0.8890157474301229 and parameters: {'lamdba_a': 4.226716726433139}. Best is trial 1 with value: 0.7268400674932671.
[I 2025-03-24 16:26:41,873] Trial 4 finished with value: 0.8893346185250165 and parameters: {'lamdba_a': 4.258103211916651}. Best is trial 1 with value: 0.7268400674932671.
[I 2025-03-24 16:26:

In [11]:
df_expo  = pd.concat([df_erc[['Exposure']], df_ferc[['Exposure']], df_aferc[['Exposure']], df_naferc[['Exposure']]], axis=1)
df_rc = pd.concat([df_erc[['Risk Contribution']], df_ferc[['Risk Contribution']], df_aferc[['Risk Contribution']], df_naferc[['Risk Contribution']]], axis=1)

df_final = pd.concat([df_expo, df_rc], axis=1)
df_final.columns = [('Exposure (%)', 'RB'), ('Exposure (%)', 'FRB'), ('Exposure (%)', 'AFRB'), ('Exposure (%)', 'NAFRB'),('Risk Contribution (%)', 'RB'), ('Risk Contribution (%)', 'FRB'), ('Risk Contribution (%)', 'AFRB'), ('Risk Contribution (%)', 'NAFRB')]
df_final.columns = pd.MultiIndex.from_tuples(df_final.columns)

df_final_asset = df_final.loc[['Asset 1','Asset 2','Asset 3','Asset 4']]
df_final_asset.loc['Asset sum'] = df_final_asset.sum()
print("Asset allocation")
(df_final_asset*100).round(2)


Asset allocation


Exposure (%)                         Risk Contribution (%)         \
                    RB     FRB    AFRB   NAFRB                    RB    FRB   
Asset 1          27.86   -6.60   18.26   13.13                  5.28  -1.05   
Asset 2          22.60   34.95   25.72   27.85                  5.28   7.93   
Asset 3          21.98    8.87   17.97   14.99                  5.28   1.90   
Asset 4          27.56   62.78   38.05   44.03                  5.28  13.38   
Asset sum       100.00  100.00  100.00  100.00                 21.13  22.16   

                         
            AFRB  NAFRB  
Asset 1     3.33   2.33  
Asset 2     6.00   6.48  
Asset 3     4.22   3.45  
Asset 4     7.63   9.02  
Asset sum  21.18  21.28

In [12]:
df_final_factor = df_final.loc[['Factor 1','Factor 2','Factor 3']]
df_final_factor.loc['Factor sum'] = df_final_factor.sum()
print("Factor allocation")
(df_final_factor*100).round(2)

Factor allocation


Exposure (%)                         Risk Contribution (%)         \
                     RB     FRB    AFRB   NAFRB                    RB    FRB   
Factor 1          98.36   93.37   96.73   95.67                 16.64   7.39   
Factor 2          20.65   26.42   22.06   22.83                  1.80   7.39   
Factor 3          37.62   42.42   39.36   40.38                  2.66   7.39   
Factor sum       156.63  162.21  158.15  158.87                 21.11  22.16   

                          
             AFRB  NAFRB  
Factor 1    13.87  12.23  
Factor 2     3.22   4.09  
Factor 3     4.08   4.94  
Factor sum  21.17  21.27